Import default tools

In [1]:
import numpy as np
import pandas as pd

Import requests, BeautifulSoup

In [2]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'html.parser')
#print(soup.prettify())

Find table

In [94]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table

Convert string to dataframe

In [95]:
lt = pd.read_html(str(My_table))
df = lt[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [96]:
df1 = df[df.Borough != 'Not assigned']

There is no table 'Not assigned' in Neighbourhood column

In [97]:
df1[df1.Neighbourhood == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [78]:
df1.shape

(103, 3)

Download csv file

In [92]:
url = "http://cocl.us/Geospatial_data"
geodf = pd.read_csv(url)
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Join latlon dataframe to neighbourhood dataframe

In [103]:
df1.join(geodf.set_index('Postal Code'), on='Postal Code')

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
